# COlumns defenition

In [1]:
CPU_SYSTEM_COL = "cpu_seconds_system"
MEMORY_SYSTEM_COL = "memory_mb_relative_system"
DISK_READ_BYTES_SYSTEM_COL = "disk_read_kb_system"
DISK_READ_COUNT_SYSTEM_COL = "disk_read_count_system"
DISK_WRITE_BYTES_SYSTEM_COL = "disk_write_kb_system"
DISK_WRITE_COUNT_SYSTEM_COL = "disk_write_count_system"
DISK_READ_TIME = "disk_read_time_ms_system"
DISK_WRITE_TIME = "disk_write_time_ms_system"
NETWORK_BYTES_SENT_SYSTEM_COL = "network_kb_sent_system"
NETWORK_PACKETS_SENT_SYSTEM_COL = "network_packets_sent_system"
NETWORK_BYTES_RECEIVED_SYSTEM_COL = "network_kb_received_system"
NETWORK_PACKETS_RECEIVED_SYSTEM_COL = "network_packets_received_system"

ENERGY_USAGE_SYSTEM_COL = "system_energy_usage_mwh"
DURATION_COL = "duration"
PC_TYPE = "PC_type"
PC_MANUFACTURER = "PC_manufacturer"
SYSTEM_FAMILY = "system_family"
PC_MODEL = "model"
MACHINE_TYPE = "machine_type"
DEVICE_NAME = "device_name"

## operating system info
OPERATING_SYSTEM = "operating_system"
OPERATING_SYSTEM_RELEASE = "operating_system_release"
OPERATING_SYSTEM_VERSION = "operating_system_version"

## cpu info
PROCESSOR_NAME = "processor_name"
PROCESSOR_PHYSICAL_CORES = "processor_physical_cores"
PROCESSOR_TOTAL_CORES = "processor_total_cores"
PROCESSOR_MAX_FREQ = "processor_max_frequency"
PROCESSOR_MIN_FREQ = "processor_min_frequency"

## RAM info
TOTAL_RAM = "total_ram"
PHYSICAL_DISK_NAME = "physical_disk_name"
PHYSICAL_DISK_MANUFACTURER = "physical_disk_manufacturer"
PHYSICAL_DISK_MODEL = "physical_disk_model"
PHYSICAL_DISK_MEDIA_TYPE = "physical_disk_media_type"
PHYSICAL_DISK_FIRMWARE_VERSION = "disk_firmware_version"
LOGICAL_DISK_NAME = "logical_disk_name"
LOGICAL_DISK_MANUFACTURER = "logical_disk_manufacturer"
LOGICAL_DISK_MODEL = "logical_disk_model"
LOGICAL_DISK_DISK_TYPE = "logical_disk_disk_type"
LOGICAL_DISK_PARTITION_STYLE = "logical_disk_partition_style"
LOGICAL_DISK_NUMBER_OF_PARTITIONS = "logical_disk_number_of_partitions"
PHYSICAL_SECTOR_SIZE = "physical_sector_size"
LOGICAL_SECTOR_SIZE = "logical_sector_size"
BUS_TYPE = "bus_type"
FILESYSTEM = "file_system"
BATTERY_DESIGN_CAPACITY = "design_battery_capacity"
FULLY_CHARGED_BATTERY_CAPACITY = "fully_charged_battery_capacity"

ENERGY_USAGE_PROCESS_COL = "energy_consumption_process_mWh"
CPU_PROCESS_COL = "cpu_usage_seconds_process"
MEMORY_PROCESS_COL = "memory_mb_relative_process"
DISK_READ_BYTES_PROCESS_COL = "disk_read_kb_process"
DISK_READ_COUNT_PROCESS_COL = "disk_read_count_process"
DISK_WRITE_BYTES_PROCESS_COL = "disk_write_kb_process"
DISK_WRITE_COUNT_PROCESS_COL = "disk_write_count_process"
PAGE_FAULT_PROCESS_COL = "number_of_page_faults_process"
NETWORK_BYTES_SENT_PROCESS_COL = "network_kb_sent_process"
NETWORK_PACKETS_SENT_PROCESS_COL = "network_packets_sent_process"
NETWORK_BYTES_RECEIVED_PROCESS_COL = "network_kb_received_process"
NETWORK_PACKETS_RECEIVED_PROCESS_COL = "network_packets_received_process"
PROCESS_ID_COL = "pid"

In [2]:
NON_NEGATIVE_COLS = [
    DURATION_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    CPU_SYSTEM_COL,
    ENERGY_USAGE_SYSTEM_COL
]

OUTLIER_COLS = [
    DURATION_COL,
    MEMORY_SYSTEM_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    CPU_SYSTEM_COL
]

PROCESS_ONLY_COLUMNS = [CPU_PROCESS_COL,
                        MEMORY_PROCESS_COL,
                        DISK_READ_BYTES_PROCESS_COL,
                        DISK_READ_COUNT_PROCESS_COL,
                        DISK_WRITE_BYTES_PROCESS_COL,
                        DISK_WRITE_COUNT_PROCESS_COL,
                        NETWORK_BYTES_SENT_PROCESS_COL,
                        NETWORK_PACKETS_SENT_PROCESS_COL,
                        NETWORK_BYTES_RECEIVED_PROCESS_COL,
                        NETWORK_PACKETS_RECEIVED_PROCESS_COL,
                        PAGE_FAULT_PROCESS_COL]

SYSTEM_ONLY_COLUMNS = [
    CPU_SYSTEM_COL,
    MEMORY_SYSTEM_COL,
    DISK_READ_BYTES_SYSTEM_COL,
    DISK_READ_COUNT_SYSTEM_COL,
    DISK_WRITE_BYTES_SYSTEM_COL,
    DISK_WRITE_COUNT_SYSTEM_COL,
    DISK_READ_TIME, DISK_WRITE_TIME,
    NETWORK_BYTES_SENT_SYSTEM_COL,
    NETWORK_PACKETS_SENT_SYSTEM_COL,
    NETWORK_BYTES_RECEIVED_SYSTEM_COL,
    NETWORK_PACKETS_RECEIVED_SYSTEM_COL,
    DURATION_COL]

# Functions

In [3]:
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
from measurements_model_pipeline.utils import calculate_and_print_scores
from sklearn.preprocessing import StandardScaler


def print_predictions(x_test, y_test, y_pred):
    for i, (actual, predicted) in enumerate(zip(y_test, y_pred)):
        print("Actual:", actual)
        print("Predicted:", predicted)
        if predicted > 100:
            print("Found bad prediction at row ", i)
            print(f"Row {i}: {x_test.iloc[i].to_dict()}")


def debug_predictions(X_test: pd.DataFrame, y_test: pd.Series, y_pred: pd.Series, top_n: int = 10):
    """
    Prints key statistics and highlights rows with the largest prediction errors
    to easily detect corrupted feature rows or prediction issues.
    """
    import pandas as pd
    import numpy as np

    # Ensure all inputs are properly formatted
    X_test = X_test.copy()
    y_test = pd.Series(y_test).reset_index(drop=True)
    y_pred = pd.Series(y_pred).reset_index(drop=True)

    # 1. General stats
    print("=== GENERAL STATS ===")
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
    print("y_pred shape:", y_pred.shape)
    print("\ny_test.describe():\n", y_test.describe())
    print("\ny_pred.describe():\n", y_pred.describe())

    # 2. Compute errors
    errors = (y_test - y_pred).abs()
    print("\nMax absolute error:", errors.max())
    print("Mean absolute error:", errors.mean())
    print("Median absolute error:", errors.median())

    # 3. Show top N largest errors
    top_errors_idx = errors.sort_values(ascending=False).head(top_n).index
    print(f"\n=== TOP {top_n} LARGEST ERRORS ===")
    for idx in top_errors_idx:
        print(f"Row {idx}:")
        print("  Features:", X_test.iloc[idx].to_dict())
        print("  Actual:", y_test.iloc[idx])
        print("  Predicted:", y_pred.iloc[idx])
        print("  Abs Error:", errors.iloc[idx])
        print("---")

    # 4. Optionally detect extreme feature values
    extreme_values = X_test[(X_test.abs() > 1e5).any(axis=1)]
    if not extreme_values.empty:
        print("\n!!! WARNING: Some feature rows have extremely large values (>1e5) !!!")
        print(extreme_values)
    else:
        print("\nNo extreme feature values detected.")


def run_pipeline(x_train, x_test, y_train, y_test):
    y_train = pd.Series(y_train.squeeze()).reset_index(drop=True)
    y_test = pd.Series(y_test.squeeze()).reset_index(drop=True)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(x_train)
    X_test_scaled = scaler.transform(x_test)

    X_train_scaled = pd.DataFrame(
        X_train_scaled,
        columns=x_train.columns,
        index=x_train.index
    )


    X_test_scaled = pd.DataFrame(
        X_test_scaled,
        columns=x_test.columns,
        index=x_test.index
    )

    model = HistGradientBoostingRegressor(max_depth=11, min_samples_leaf=10)
    model.fit(X_train_scaled, y_train)

    y_pred = pd.Series(model.predict(X_test_scaled)).reset_index(drop=True)

    print_predictions(X_test_scaled, y_test, y_pred)
    debug_predictions(X_test_scaled, y_test, y_pred, top_n=10)

    calculate_and_print_scores(y_test, y_pred)
    return scaler, model

In [4]:
import numpy as np


def process_df(df: pd.DataFrame, non_negative_cols: list[str], outlier_cols: list[str]) -> pd.DataFrame:
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    if non_negative_cols is not None:
        for col in non_negative_cols:
            df = df[df[col] >= 0]

    if outlier_cols is not None:
        for col in outlier_cols:
            low = df[col].quantile(0.01)
            high = df[col].quantile(0.99)
            df = df[(df[col] >= low) & (df[col] <= high)]

    return df

In [5]:
from sklearn.model_selection import train_test_split

def execute_model(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()
    model = run_pipeline(X_train, X_test, y_train, y_test)
    return model

# Read system only data

In [6]:
system_df = pd.read_csv("system_df.csv", index_col=0)
system_df = process_df(system_df, NON_NEGATIVE_COLS, OUTLIER_COLS)
X = system_df.drop(columns=[ENERGY_USAGE_SYSTEM_COL])
y = system_df[ENERGY_USAGE_SYSTEM_COL]

In [7]:
system_scaler, system_model = execute_model(X, y)

Actual: 46.79920792
Predicted: 46.93861710120535
Actual: 9.73175896
Predicted: 11.090239005450739
Actual: 6.639591149999999
Predicted: 6.803750549435668
Actual: 10.06432029
Predicted: 10.856837072743357
Actual: 46.57991022
Predicted: 47.47016255798545
Actual: 58.852538
Predicted: 57.08574257045556
Actual: 58.92894890666667
Predicted: 58.204271360717854
Actual: 58.03732138
Predicted: 58.16061071354229
Actual: 17.359863913333335
Predicted: 13.293091677866617
Actual: 58.60084044
Predicted: 58.00133176793917
Actual: 5.104683386666667
Predicted: 13.052083520051864
Actual: 15.107442386666667
Predicted: 12.484069152471985
Actual: 57.90747493999999
Predicted: 57.511578311084016
Actual: 6.80002704
Predicted: 7.013047994528434
Actual: 57.85710318
Predicted: 57.810008742055615
Actual: 10.422580773333332
Predicted: 10.676607980856227
Actual: 10.74159607
Predicted: 11.597850145777091
Actual: 17.383713123333333
Predicted: 13.343071392038386
Actual: 12.790619799999998
Predicted: 11.476956026095761
Ac

# Read Process Data

In [8]:
process_system_df = pd.read_csv("process_system_df.csv", index_col=0)

## Get System - Process df

get a dataframe with the substruction of the total system and the specific process.

In [9]:
process_system_df.head()

,duration,cpu_usage_seconds_process,memory_mb_relative_process,disk_read_kb_process,disk_read_count_process,disk_write_kb_process,disk_write_count_process,number_of_page_faults_process,network_kb_sent_process,network_packets_sent_process,...,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_sent_system,network_packets_sent_system,network_kb_received_system,network_packets_received_system,energy_consumption_process_mWh
0,5.260686,0.0,0.574,0.0,0,0.0,0,147,0.0,0,...,37,6674.5,408,0,0,0.700,8,17,17,2.665414
1,5.318386,0.0,-0.238,0.0,0,0.0,0,1160,0.0,0,...,3,1439.5,102,0,0,3.155,5,7,7,2.694649
2,5.249743,0.0,0.258,0.0,0,0.0,0,66,0.0,0,...,9,1062.5,63,0,0,8.820,108,6,6,2.659870
3,5.307450,0.0,0.250,0.0,0,0.0,0,64,0.0,0,...,13,1307.5,101,0,0,0.000,0,17,17,2.689108
4,5.242309,0.0,0.199,0.0,0,0.0,0,484,0.0,0,...,2,1965.5,37,0,0,70.788,120,35,35,2.656103


In [10]:
process_only_df = process_system_df[PROCESS_ONLY_COLUMNS].copy()
system_only_df = system_df[SYSTEM_ONLY_COLUMNS].copy()


In [11]:
system_only_df[CPU_SYSTEM_COL] = system_only_df[CPU_SYSTEM_COL] - process_only_df[CPU_PROCESS_COL]
system_only_df[MEMORY_SYSTEM_COL] = system_only_df[MEMORY_SYSTEM_COL] - process_only_df[MEMORY_PROCESS_COL]
system_only_df[DISK_READ_BYTES_SYSTEM_COL] = system_only_df[DISK_READ_BYTES_SYSTEM_COL] - process_only_df[DISK_READ_BYTES_PROCESS_COL]
system_only_df[DISK_READ_COUNT_SYSTEM_COL] = system_only_df[DISK_READ_COUNT_SYSTEM_COL] - process_only_df[DISK_READ_COUNT_PROCESS_COL]
system_only_df[DISK_WRITE_BYTES_SYSTEM_COL] = system_only_df[DISK_WRITE_BYTES_SYSTEM_COL] - process_only_df[DISK_WRITE_BYTES_PROCESS_COL]
system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] = system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] - process_only_df[DISK_WRITE_COUNT_PROCESS_COL]
system_only_df[NETWORK_BYTES_SENT_SYSTEM_COL] = system_only_df[NETWORK_BYTES_SENT_SYSTEM_COL] - process_only_df[NETWORK_BYTES_SENT_PROCESS_COL]
system_only_df[NETWORK_PACKETS_SENT_SYSTEM_COL] = system_only_df[NETWORK_PACKETS_SENT_SYSTEM_COL] - process_only_df[NETWORK_PACKETS_SENT_PROCESS_COL]
system_only_df[NETWORK_BYTES_RECEIVED_SYSTEM_COL] = system_only_df[NETWORK_BYTES_RECEIVED_SYSTEM_COL] - process_only_df[NETWORK_BYTES_RECEIVED_PROCESS_COL]
system_only_df[NETWORK_PACKETS_RECEIVED_SYSTEM_COL] = system_only_df[NETWORK_PACKETS_RECEIVED_SYSTEM_COL] - process_only_df[NETWORK_PACKETS_RECEIVED_PROCESS_COL]
system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] = system_only_df[DISK_WRITE_COUNT_SYSTEM_COL] - process_only_df[DISK_WRITE_COUNT_PROCESS_COL]


In [12]:
system_only_df

,cpu_seconds_system,memory_mb_relative_system,disk_read_kb_system,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_sent_system,network_packets_sent_system,network_kb_received_system,network_packets_received_system,duration
0,4.192767,30.146,692.0,37.0,6674.5,408.0,0,0,0.700,8.0,17.0,17.0,5.260686
1,3.736166,3.310,36.0,3.0,1439.5,102.0,0,0,3.155,5.0,7.0,7.0,5.318386
2,3.260090,-3.330,100.0,9.0,1062.5,63.0,0,0,8.820,108.0,6.0,6.0,5.249743
4,3.242368,1.849,68.0,2.0,1965.5,37.0,0,0,70.788,120.0,35.0,35.0,5.242309
5,3.836937,-6.277,28.0,3.0,1988.0,95.0,0,0,0.707,8.0,3.0,3.0,5.277767
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,9.153270,12.288,3021.5,99.0,1552.5,134.0,1,0,12.570,43.0,45.0,45.0,2.153204
8124,11.195836,-6.144,16280.0,1880.0,8616.0,33.0,0,0,1.102,14.0,12.0,12.0,2.299648
8130,7.529029,-5.120,108.0,8.0,1922.0,259.0,0,0,1.380,15.0,17.0,17.0,2.152381
8179,10.522279,-18.432,128.0,6.0,2607.5,56.0,0,0,6.616,19.0,26.0,26.0,2.132174


In [13]:
system_df.columns.to_list()[:-1] == system_only_df.columns.to_list()

True

In [17]:
system_only_scaled = system_scaler.transform(system_only_df)
system_only_df_scaled = pd.DataFrame(
        system_only_scaled,
        columns=system_only_df.columns,
        index=system_only_df.index
    )

In [18]:
system_only_df_scaled

,cpu_seconds_system,memory_mb_relative_system,disk_read_kb_system,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_sent_system,network_packets_sent_system,network_kb_received_system,network_packets_received_system,duration
0,-0.881934,1.622545,0.142781,0.041270,3.970453,2.263057,-0.142321,-0.253526,-0.655402,-1.241130,-0.365010,-0.365010,-0.728894
1,-0.916712,0.135149,-0.275233,-0.177762,-0.530852,-0.571003,-0.142321,-0.253526,-0.451603,-1.298964,-0.822073,-0.822073,-0.709973
2,-0.952973,-0.232876,-0.234451,-0.139109,-0.855015,-0.932206,-0.142321,-0.253526,0.018670,0.686673,-0.867780,-0.867780,-0.732482
4,-0.954323,0.054172,-0.254842,-0.184204,-0.078572,-1.173009,-0.142321,-0.253526,5.162868,0.918009,0.457704,0.457704,-0.734920
5,-0.909037,-0.396215,-0.280331,-0.177762,-0.059226,-0.635834,-0.142321,-0.253526,-0.654821,-1.241130,-1.004898,-1.004898,-0.723293
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,-0.504110,0.632758,1.627176,0.440681,-0.433690,-0.274630,7.026361,-0.253526,0.329972,-0.566399,0.914767,0.914767,-1.747862
8124,-0.348535,-0.388843,10.075706,11.914083,5.639849,-1.210055,-0.142321,-0.253526,-0.622030,-1.125462,-0.593542,-0.593542,-1.699842
8130,-0.627823,-0.332088,-0.229353,-0.145552,-0.115976,0.883074,-0.142321,-0.253526,-0.598952,-1.106184,-0.365010,-0.365010,-1.748132
8179,-0.399838,-1.069911,-0.216609,-0.158436,0.473450,-0.997038,-0.142321,-0.253526,-0.164292,-1.029071,0.046347,0.046347,-1.754758


In [19]:
energy_predictions_system_without_process = system_model.predict(system_only_df_scaled)

In [20]:
energy_predictions_system_without_process

array([10.56732808, 10.06670031,  9.68112551, ...,  6.97663414,
        6.85188597,  6.86449988], shape=(7493,))

In [ ]:
system_df[ENERGY_USAGE_SYSTEM_COL]

In [21]:
process_energy_predictions = system_df[ENERGY_USAGE_SYSTEM_COL] - energy_predictions_system_without_process

In [29]:
system_only_df_scaled.shape

(7493, 13)

In [26]:
system_df[ENERGY_USAGE_SYSTEM_COL].shape

(7493,)

In [23]:
process_system_df[ENERGY_USAGE_PROCESS_COL] = process_energy_predictions

In [24]:
process_system_df

,duration,cpu_usage_seconds_process,memory_mb_relative_process,disk_read_kb_process,disk_read_count_process,disk_write_kb_process,disk_write_count_process,number_of_page_faults_process,network_kb_sent_process,network_packets_sent_process,...,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_sent_system,network_packets_sent_system,network_kb_received_system,network_packets_received_system,energy_consumption_process_mWh
0,5.260686,0.0,0.574,0.0,0,0.0,0,147,0.0,0,...,37,6674.5,408,0,0,0.700,8,17,17,0.357363
1,5.318386,0.0,-0.238,0.0,0,0.0,0,1160,0.0,0,...,3,1439.5,102,0,0,3.155,5,7,7,0.977815
2,5.249743,0.0,0.258,0.0,0,0.0,0,66,0.0,0,...,9,1062.5,63,0,0,8.820,108,6,6,1.220841
3,5.307450,0.0,0.250,0.0,0,0.0,0,64,0.0,0,...,13,1307.5,101,0,0,0.000,0,17,17,NaN
4,5.242309,0.0,0.199,0.0,0,0.0,0,484,0.0,0,...,2,1965.5,37,0,0,70.788,120,35,35,0.588193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8178,2.083399,0.0,0.000,0.0,0,0.0,0,0,0.0,0,...,17,660.0,80,0,0,10.759,58,74,74,NaN
8179,2.132174,0.0,0.000,0.0,0,0.0,0,0,0.0,0,...,6,2607.5,56,0,0,6.616,19,26,26,0.603616
8180,2.088345,0.0,0.000,0.0,0,0.0,0,0,0.0,0,...,6,914.5,33,0,0,1.760,20,32,32,NaN
8181,2.143558,0.0,0.000,0.0,0,0.0,0,0,0.0,0,...,16,740.5,59,0,0,7.461,19,22,22,0.630808
